CONCLUSIONS AT THE END. VERY QUITE INTERESTING NOTEBOOK HOWEVER.

This is inspired in the Otto product clasification Challenge, and on this kernel: [https://www.kaggle.com/pratik2901/otto-product](http://)
The goal is to classify the products with a random forest and then compare the results with other methods. KNN, SVM, etc, just to play and learn a bit. NO CROSS VALIDATION IN THE RANDOM FOREST FOR SIMPLICITY. IN THE OTHERS I DO IT.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle

print("All imports OK")

**READ DATA**

In [ ]:
%%time
#As usual, I get confused with the kaggle input. Just ../input/train_file.csv !!
#There is no target column in the test file, so I need to split my train and use part to validate
train=pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")
print("Everything looks fine so far")

Now, it is possible to use the training data "as is" or I can shuffle the rows randomly. If it's not necessary to shuffle, the result won't change, but if the Otto people didn't produce the data with a random order it may impact the clasification. I'll do it because I can.

ALSO, A MATRIX PLOT IN THIS CASE WOULD BE HUGE, TOO MANY VARIABLES.

In [ ]:
train=shuffle(train)
train.head()
#Notice the id is index+1

Now I´ll se how many products I have per class

In [ ]:
#I want at the beginning of the bar plot: class 1, then 2, etc.
ordered=sorted(set(train['target']))

sns.countplot(x='target', data=train, order=ordered);

plt.title("Number of products per class");
plt.figure();


In [ ]:
#check that there are no missing values
print(train.isnull().sum().sum())
#Double sum, to sum over all the list of rows. If it is not 0, remove on of the .sum() and find where they NaNs are.

Ok, there are many producs of class 2 and 6. I am not sure that will be problematic, just that other classes may not be classified as accurately.

Besides, scikitlearn doesn't like class_1 or so as labels. Need encoding and in this case I'll do only label encoding. One-hot encoding adds dimensions and I'll do it at some point in the very near future

In [ ]:
X=train
Y=train.target
X=X.drop(['target','id'], axis=1)

from sklearn import preprocessing
LE=preprocessing.LabelEncoder()
LE.fit_transform(Y.values.tolist())
Labels=LE.transform(Y.astype(str))
print(LE.classes_)
print(Labels)

Since I had a look at the head of the train/test files, and they are very sparse, I am sure I can remove some features while not losing too much info, because they don't contribute to the model too much. Like an mp3 or jpg encoding. This also reduces computation time, which will be significant with large datasets.

RFE is recursive feature elimination. scikitlearn does a lot of things easilty for me, so let's use it. For more basic info:

[https://medium.com/@aneesha/recursive-feature-elimination-with-scikit-learn-3a2cbdf23fb7](http://)

and

[https://www.kaggle.com/arthurtok/feature-ranking-rfe-random-forest-linear-models](http://)

In [ ]:
%%time

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

n_feat=30 #I´ll play a bit with this value and leave the one I like the most (45,30,15)
#The selection takes the same amount of time with 1 feature or with 100, it ranks all of htem and then selects the best ones
#45 is a lot but better than 30, you lose too much info. I'll put 30 for speed.

clasif=RandomForestClassifier()
rfe=RFE(clasif,n_feat)

fit=rfe.fit(X, Labels)

print("Num features: %d" %fit.n_features_)
print("Selected features: %s" %fit.support_)
print("Feature ranking: %s" %fit.ranking_) #Best is 1, worse is increasing

features=[]

#Now I´ll do a loop. If my feature is retained (==True), I keep it.
for i,j in zip(X.columns,fit.support_): #zip iterates on two lists in parallell, not only in one.
    if j==True:
        features.append(str(i))



I know which features to keep and now I will classify them. Train and test with the RFC

**FULL DETAILS OF TUNING THE RFC AND ITS PARAMETERS HERE:**

[https://medium.com/@taplapinger/tuning-a-random-forest-classifier-1b252d1dde92](http://)

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html](http://)

In [ ]:
%%time

X_RFC=X[features]
X_RFC.head() #As always, to check
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y=train_test_split(X_RFC,Y,random_state=13)

#Now I train my model with my data (No splitting necessary, it was already split in train and test files)

model_rfc=RandomForestClassifier(n_jobs=1, max_depth=13, random_state=17)
#I can add more parameters, but the default should be ok in this set
#I didn´t touch Y, X_smt I will keep on changing
model_rfc.fit(train_X,train_y)

pred_RFC=model_rfc.predict(val_X)


Now I evaluate my model. This is not a numerical regressor, so MAE and RMSE are a bit difficult to understand in this case. I need accuracy_score and confusion matrix

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

acc_RFC=np.round(accuracy_score(val_y,pred_RFC),4)
print("Accuracy RFC is: ", acc_RFC)

Not bad, 77% accurate. Let's see what is the modal category, in case I'm not predicting very much.

In [ ]:
mode=train_y.mode()
ls=[mode for i in range(len(val_y))]
#ls
acc_mode=np.round(accuracy_score(val_y,ls),4)
print("Accuracy With mode is: ", acc_mode)

My random forest predicts quite well. Let´s do the confusion matrix and the AOC curve

Bibliography for CM here: [https://www.python-course.eu/confusion_matrix.php](http://)

In [ ]:
labels = [sorted(train.target.unique())]
labels
cm = confusion_matrix(val_y, pred_RFC)
print(cm)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

My CM is not great, but not too bad. You can see that CM(0,0) is blue, which is more or less bad, but there were not too many elements in class 1. Whites are NaNs.

In [ ]:
cm_N = cm/cm.sum(axis=1)
print(cm_N)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
cax_N = ax.matshow(cm_N)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax_N)
ax.set_xticklabels([''] + labels)
#ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

sklearn does not support multiclass ROC/AUC metrics, so it is necessary to make it myself.

FOR REFERENCE
* PRECISION = DEFINITION IN ELECTRICAL PROTECTIONS. HOW MANY TRUE POSITIVES OVER TOTAL POSITIVES ------>1 IF MY CLASIFFIER/PROTECTION WORKS PROPERLY

* RECALL= TRUE POSITIVES/(TRUE POSITIVES+FALSE NEGATIVES). THIS HAD ANOTHER NAME IN ELECTRICAL PROTECTIONS BUT I DON'T REMEMBER IT. BASICALLY, I WANT FALSE NEGATIVES TO BE ZERO, SO ------>1

FOR MULTICLASS, I SUM OVER COLUMNS (AXIS =1) FOR RECALL AND I SUM OVER ROWS FOR PRECISION (AXIS=0)



In [ ]:
recall_RFC=np.diag(cm)/np.sum(cm, axis=1)
#recall_RFC
precision_RFC=np.diag(cm)/np.sum(cm, axis=0)
#precision_RFC

Now, I can either average everything normally or do the weighted average, which makes more sense when the classes are unbalanced. However, I dropped the target

In [ ]:
#Calculate the weights

total_prod=len(train_X.index)
total_classes=train.groupby('target').size() #size() gives a list, count() gives a df
#total_classes
weights=0.75*total_classes/total_prod
#The 0.75 is because I´m doing the weights with the FULL train set, not with the train. The proportions should be the same, as you would expect if the splitting is random
#Then when I divide by the training set, result is > 1, which I correct with 0.75. It´s not a great solution, but it´s a solution.
#Anyway, the numbers are all irrelevant, this is not a real case.


In [ ]:
av_recall_RFC=np.multiply(recall_RFC,weights).sum() #each*weight, and then sum all
av_prec_RFC=np.multiply(precision_RFC,weights).sum()

print("The average recall of the RFC is: ", av_recall_RFC)
print("The average precision of the RFC is: ", av_prec_RFC)

That means my recall (TP/TP+FN) has a 27% error when my precision (TP/TP+FP) is 75%. I get too many false negatives and false positives. Let's see the F1 score (valor F in "espanish"). Here I will set a value of $\beta=1$ because in this case I care as much about precision as about recall. Otherwise I adjust it accordingly.
[https://en.wikipedia.org/wiki/F1_score](http://)

In [ ]:
beta=1
F1_RFC=(1+beta)*(av_recall_RFC*av_prec_RFC)/(beta**2*av_prec_RFC+av_recall_RFC)
print("The F1 score for the RFC is: ",F1_RFC)

Now I´ll implement a kNN classifier for the multiclass problem. I USE CROSS VALIDATION. I don't do train_test_split because it was done previously, same as with label encoding.

In [ ]:
%%time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score
#I do with 3 neighbors, then I do a loop with many for cross val

knn=KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn.fit(train_X,train_y)
pred_KNN_3=knn.predict(val_X)
#For completeness, I use the built in metrics, later I'll calculate the same thing as before, for consistency
acc_KNN_3=accuracy_score(val_y, pred_KNN_3)
print("The accuracy score from sklearn for KNN3 is: ",acc_KNN_3)




75% percent "pseudo-accuracy". Wow, this is powerful but very slow. I'll do a loop for cross-validation and find the best K. Try not to run this loop so much, it's a very slow loop although it's 10 iterations for K and 5 cross-validations. THIS TAKES SOME TIME AND A CRAZY AMOUNT OF MEMORY, COMPARED TO OTHER CLASSIFIERS. Logical, to be honest.

In [ ]:
%%time

from sklearn.model_selection import cross_val_score
values_K = list(range(1,15)) #More than 10 neighbours is a bit too much

# empty list for scores
cv_scores = []

#I use accuracy as a metric. I have to choose something and then I'll do a proper fitting and cm and all that with the optimum.


# 5-fold cross validation
for k in values_K:
    knn = KNeighborsClassifier(n_neighbors=k,n_jobs=-1)
    scores = cross_val_score(knn, train_X, train_y, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

print("Finished!")

Let's print the score for every K. The best one is for 9 neighbors. I may have to do a few more, but the iterations take very long. I won't plot the scatter because there are 9 classes, that's a bit much.

In [ ]:
print("The optimum number of neighbors is: ", cv_scores.index(max(cv_scores))+1) #Because Index starts at 0!!!
plt.plot(values_K, cv_scores)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Accuracy score')
plt.show()

In [ ]:
%%time

model_knn=KNeighborsClassifier(n_neighbors=9,n_jobs=-1)
model_knn.fit(train_X,train_y)

pred_KNN_9=model_knn.predict(val_X)
#Now I do everything with confusion matrix, as before.

print("Trained and predicted")


In [ ]:
cm_knn = confusion_matrix(val_y, pred_KNN_9)
print(cm_knn)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
cax = ax.matshow(cm_knn)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
recall_KNN=np.diag(cm_knn)/np.sum(cm_knn, axis=1)
#recall_RFC
precision_KNN=np.diag(cm_knn)/np.sum(cm_knn, axis=0)
#precision_RFC

In [ ]:
av_recall_KNN=np.multiply(recall_KNN,weights).sum() #each*weight, and then sum all
av_precision_KNN=np.multiply(precision_KNN,weights).sum()

print("The average recall of the RFC is: ", av_recall_KNN)
print("The average precision of the RFC is: ", av_precision_KNN)

Now let's do a gradient boosted decision tree. THIS IS NOT A RANDOM FOREST. An RF builds many trees and combines them (average for example). GBT is a SEQUENTIAL set of trees. Build 1, Improve! (tune param) and then build a second one. Improve the second one, and so on. IT'S SEQUENTIAL, NOT PARALLELL. In this case, it will be much slower, but perhaps better in accuracy. Again, I don't need to split. I did it previously.

THE NEXT CELL TAKES VERY LONG, ~7 min.

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

model_GBDT=GradientBoostingClassifier(loss='deviance', learning_rate=0.15, n_estimators=100, subsample=1, max_depth=8, random_state=13) #No n_jobs
model_GBDT.fit(train_X,train_y)
pred_GBDT=model_GBDT.predict(val_X)
#print("Model fitted and predictions done.")
#train_y.head()



In [ ]:
cm_GBDT = confusion_matrix(val_y, pred_GBDT)
#print(cm_GBDT)
#fig = plt.figure(figsize=(20,10))
#ax = fig.add_subplot(111)
#cax = ax.matshow(cm_GBDT)
#plt.title('Confusion matrix of the classifier')
#fig.colorbar(cax)
#ax.set_xticklabels([''] + labels)
#ax.set_yticklabels([''] + labels)
#plt.xlabel('Predicted')
#plt.ylabel('True')
#plt.show()

In [ ]:
recall_GBDT=np.diag(cm_GBDT)/np.sum(cm_GBDT, axis=1)
#recall_RFC
precision_GBDT=np.diag(cm_GBDT)/np.sum(cm_GBDT, axis=0)
#precision_RFC

av_recall_GBDT=np.multiply(recall_GBDT,weights).sum() #each*weight, and then sum all
av_precision_GBDT=np.multiply(precision_GBDT,weights).sum()

print("The average recall of the RFC is: ", av_recall_GBDT)
print("The average precision of the RFC is: ", av_precision_GBDT)

The GBDT is the best so far. Let's see a (few) SVMs. Basically, linear, polynomial low-high order, rbf. KEEP IN MIND THE SVM ARE VERY BAD FOR SCALING, THEY RUN IN O(N^2) FOR N SAMPLES. IN PRACTICE MORE THAN 10-20 K SAMPLES ARE NOT VERY CLASSIFIABLE.
[https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html](http://)

I'm going to save work and do a few at the same time, then compare.

In [ ]:
%%time

from sklearn import svm

model_SVC_lin=svm.SVC(C=1, kernel='linear')
model_SVC_lin.fit(train_X,train_y)
pred_SVC_lin=model_SVC_lin.predict(val_X)

cm_SVC_lin = confusion_matrix(val_y, pred_SVC_lin)

recall_SVC_lin=np.diag(cm_SVC_lin)/np.sum(cm_SVC_lin, axis=1)
#recall_RFC
precision_SVC_lin=np.diag(cm_SVC_lin)/np.sum(cm_SVC_lin, axis=0)
#precision_RFC

av_recall_SVC_lin=np.multiply(recall_SVC_lin,weights).sum() #each*weight, and then sum all
av_precision_SVC_lin=np.multiply(precision_SVC_lin,weights).sum()

print("The average recall of the SVC with linear kernel is: ", av_recall_SVC_lin)
print("The average precision of the SVC with linear kernel is: ", av_precision_SVC_lin)




Ok, this is not great, but similar to the other ones. Which is not surprising, it's a linear clasification and can be a bit coarse.

In [ ]:
%%time
#polynomial. I'll use degree 3-5 and decide manually


model_SVC_p=svm.SVC(C=1, kernel='poly', degree=3)
model_SVC_p.fit(train_X,train_y)
pred_SVC_p=model_SVC_p.predict(val_X)

cm_SVC_p = confusion_matrix(val_y, pred_SVC_p)

recall_SVC_p=np.diag(cm_SVC_p)/np.sum(cm_SVC_p, axis=1)
#recall_RFC
precision_SVC_p=np.diag(cm_SVC_p)/np.sum(cm_SVC_p, axis=0)
#precision_RFC

av_recall_SVC_p=np.multiply(recall_SVC_p,weights).sum() #each*weight, and then sum all
av_precision_SVC_p=np.multiply(precision_SVC_p,weights).sum()

print("The average recall of the SVC of degree 3 is: ", av_recall_SVC_p)
print("The average precision of the SVC of degree 3 is: ", av_precision_SVC_p)

#third degree: recall/prec = 0.751/0.751



In [ ]:
#%%time
#polynomial. I'll use degree 3-5 and decide manually


#model_SVC_p=svm.SVC(C=1, kernel='poly', degree=5)
#model_SVC_p.fit(train_X,train_y)
#pred_SVC_p=model_SVC_p.predict(val_X)

#cm_SVC_p = confusion_matrix(val_y, pred_SVC_p)

#recall_SVC_p=np.diag(cm_SVC_p)/np.sum(cm_SVC_p, axis=1)
#recall_RFC
#precision_SVC_p=np.diag(cm_SVC_p)/np.sum(cm_SVC_p, axis=0)
#precision_RFC

#av_recall_SVC_p=np.multiply(recall_SVC_p,weights).sum() #each*weight, and then sum all
#av_precision_SVC_p=np.multiply(precision_SVC_p,weights).sum()

#print("The average recall of the SVC of degree 5 is: ", av_recall_SVC_p)
#print("The average precision of the SVC of degree 5 is: ", av_precision_SVC_p)

#third degree: recall/prec = 0.751/0.751

In [ ]:
%%time
#RBF

model_SVC_rbf=svm.SVC(C=1, kernel='rbf')
model_SVC_rbf.fit(train_X,train_y)
pred_SVC_rbf=model_SVC_rbf.predict(val_X)

cm_SVC_rbf = confusion_matrix(val_y, pred_SVC_rbf)

recall_SVC_rbf=np.diag(cm_SVC_rbf)/np.sum(cm_SVC_rbf, axis=1)
#recall_RFC
precision_SVC_rbf=np.diag(cm_SVC_p)/np.sum(cm_SVC_rbf, axis=0)
#precision_RFC

av_recall_SVC_rbf=np.multiply(recall_SVC_rbf,weights).sum() #each*weight, and then sum all
av_precision_SVC_rbf=np.multiply(precision_SVC_rbf,weights).sum()

print("The average recall of the SVC with an rbf kernel is: ", av_recall_SVC_rbf)
print("The average precision of the SVC with an rbf kernel is: ", av_precision_SVC_rbf)



I'm not going to do a plot of times because I already took too long with this thing. However, SVM with linear kernel is ~5 min and GDBT is ~8 min. However, SVC with 3rd degree kernel is 23 min!!!!! I didn't bother to try the 5th degree anyway.

In [ ]:
F1_RFC=round((1+beta)*(av_recall_RFC*av_prec_RFC)/(beta**2*av_prec_RFC+av_recall_RFC),4)
print("The F1 score for the RFC is: ",F1_RFC)

F1_KNN_9=round((1+beta)*(av_recall_KNN*av_precision_KNN)/(beta**2*av_precision_KNN+av_recall_KNN),4)
print("The F1 score for the KNN with 9 neighbors is: ",F1_KNN_9)

F1_GBDT=round((1+beta)*(av_recall_GBDT*av_precision_GBDT)/(beta**2*av_precision_GBDT+av_recall_GBDT),4)
print("The F1 score for the GBDT is: ",F1_GBDT)


F1_SVC_lin=round((1+beta)*(av_recall_SVC_lin*av_precision_SVC_lin)/(beta**2*av_precision_SVC_lin+av_recall_SVC_lin),4)
print("The F1 score for the SVM with linear kernel is: ",F1_SVC_lin)

F1_SVC_p=round((1+beta)*(av_recall_SVC_p*av_precision_SVC_p)/(beta**2*av_precision_SVC_p+av_recall_SVC_p),4)
print("The F1 score for the SVM with 3rd degree kernel is: ",F1_SVC_p)

F1_SVC_rbf=round((1+beta)*(av_recall_SVC_rbf*av_precision_SVC_rbf)/(beta**2*av_precision_SVC_rbf+av_recall_SVC_rbf),4)
print("The F1 score for the SVM with 3rd degree kernel is: ",F1_SVC_rbf)




Conclusion: GBDT is the best in precision. Significantly higher than the other ones. Now, with RFE I reduce times and memory, but of course I'll lose some info. Can be a good strategy to play with and then do a finer model. Avoid SVM with mid to big datasets, especially if you need to try a different polynomial model. Perhaps rbf is better than polynomial, or linear. Depends on the data. Random forest is still the fastest one because it creates the trees in a parallell way, while GDBT are sequential.